<a href="https://colab.research.google.com/github/kaitoyama/enigmaBombe/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U networkx
!apt install graphviz libgraphviz-dev
!pip install pygraphviz

In [2]:
import networkx as nx
import itertools
import tqdm

In [3]:
roter_1=["R",['E', 'K', 'M', 'F', 'L', 'G', 'D', 'Q', 'V', 'Z', 'N', 'T', 'O', 'W', 'Y', 'H', 'X', 'U', 'S', 'P', 'A', 'I', 'B', 'R', 'C', 'J']]
roter_2=["F",['A', 'J', 'D', 'K', 'S', 'I', 'R', 'U', 'X', 'B', 'L', 'H', 'W', 'T', 'M', 'C', 'Q', 'G', 'Z', 'N', 'P', 'Y', 'F', 'V', 'O', 'E']]
roter_3=["W",['B', 'D', 'F', 'H', 'J', 'L', 'C', 'P', 'R', 'T', 'X', 'V', 'Z', 'N', 'Y', 'E', 'I', 'W', 'G', 'A', 'K', 'M', 'U', 'S', 'Q', 'O']]
UKW_B=['Y', 'R', 'U', 'H', 'Q', 'S', 'L', 'D', 'P', 'X', 'N', 'G', 'O', 'K', 'M', 'I', 'E', 'B', 'F', 'Z', 'C', 'W', 'V', 'J', 'A', 'T']

In [4]:
def alphabet2num(alphabet: str):
    upperAlpha=alphabet.upper()
    return(ord(upperAlpha)-65)

def num2alphabet(num: int):
    return(chr(num+65))

def num2alpha(num: int):
    if num<=26:
        return chr(64+num)
    elif num%26==0:
        return num2alpha(num//26-1)+chr(90)
    else:
        return num2alpha(num//26)+chr(64+num%26)

In [5]:
def justify(order: int):
    if 0<=order<26:
        return order
    elif order<0:
        while order<0:
            order+=26
        return order
    else:
        while 25<order:
            order-=26
        return order

In [6]:
def enigmaEncode(input: str,roter: list,position: list,Ringstellung: list,plugboard={},offset=0):
    if len(plugboard)!=0:
        for i in plugboard.keys():
            input=input.replace(plugboard[i],"-")
            input=input.replace(i,plugboard[i])
            input=input.replace("-",i)
    enc=[]
    secondTurn=0
    thirdTurn=0
    if offset>0:
        for i in range(offset):
            firstDif=justify(alphabet2num(position[0])+alphabet2num(Ringstellung[0])+i)
            if justify(alphabet2num(position[0])+i)==alphabet2num(roter[0][0]):
                secondTurn+=1
            secondDif=justify(alphabet2num(position[1])+alphabet2num(Ringstellung[1])+secondTurn)
            if justify(alphabet2num(position[1])+secondTurn)==alphabet2num(roter[1][0]):
                thirdTurn+=1
    elif offset<0:
        for i in range(abs(offset)):
            firstDif=justify(alphabet2num(position[0])+alphabet2num(Ringstellung[0])-i)
            if justify(alphabet2num(position[0])-i)==alphabet2num(roter[0][0]):
                secondTurn-=1
            secondDif=justify(alphabet2num(position[1])+alphabet2num(Ringstellung[1])+secondTurn)
            if justify(alphabet2num(position[1])+secondTurn)==alphabet2num(roter[1][0]):
                thirdTurn-=1
    for i in range(len(input)):
        current=input[i]
        i+=1
        firstDif=justify(alphabet2num(position[0])+alphabet2num(Ringstellung[0])+offset+i)
        if justify(alphabet2num(position[0])+offset+i)==alphabet2num(roter[0][0]):
            secondTurn+=1
        secondDif=justify(alphabet2num(position[1])+alphabet2num(Ringstellung[1])+secondTurn)
        if justify(alphabet2num(position[1])+secondTurn)==alphabet2num(roter[1][0]):
            thirdTurn+=1       
        thirdDif=justify(alphabet2num(position[2])+alphabet2num(Ringstellung[2])+thirdTurn)
        current=roter[0][1][justify(alphabet2num(current)+firstDif)]
        current=roter[1][1][justify(alphabet2num(current)+secondDif)]
        current=roter[2][1][justify(alphabet2num(current)+thirdDif)]
        current=UKW_B[alphabet2num(current)]
        current=num2alphabet(justify(roter[2][1].index(current)-thirdDif))
        current=num2alphabet(justify(roter[1][1].index(current)-secondDif))
        current=num2alphabet(justify(roter[0][1].index(current)-firstDif))
        enc.append(current)
    return("".join(enc))

In [ ]:
enigmaEncode("CONSEQUATINVIDUNTREBUMLUPTATUMVULPUTATESITLOREMREBUMFACILISISSANCTUSLABORESEALOREMTEMPORLOREMKASDDOLORDOLORSEDCONSECTETUER",[roter_2,roter_1,roter_3],["A","B","J"],["A","A","B"],{"a":"d","c":"r","k":"l"})

In [8]:
enc = "LLKWYIQNPPRHVZOTJLIOLZYBXHWGYBWVWJJIJVKKCAEGGMKSDMQIXTEEODZLMTCUVIZZMNHVCYLFHNHOTPNUPZYFIYEKAUNIANRBYDRUIALPAZYYXUJNJPCICN"#@param {type:"string"}
plain="CONSEQUATINVIDUNTREBUMLUPTATUMVULPUTATESITLOREMREBUMFACILISISSANCTUSLABORESEALOREMTEMPORLOREMKASDDOLORDOLORSEDCONSECTETUER"#@param{type:"string"}

In [ ]:
from IPython.display import SVG, display

G = nx.DiGraph()
crib_list=[plain,enc]
for i in range(len(crib_list[0])):
    nx.add_path(G, [crib_list[0][i],crib_list[1][i]],weight=i+1)

svg = SVG(nx.nx_agraph.to_agraph(G).draw(prog='fdp', format='svg'))
display(svg)

In [10]:
cycleList=sorted(nx.simple_cycles(G))

In [11]:
checker=0
for i in range(len(cycleList)):
    if len(cycleList[checker])<len(cycleList[i]):
        checker=i
maxCycle=cycleList[checker]

In [ ]:
maxCycle

In [ ]:
weightList=[]
for i in range(len(maxCycle)):
    target=sorted(G.edges(maxCycle[i],data=True))
    if i!=len(maxCycle)-1:
        for j in range(len(target)):
            if target[j][1]==maxCycle[i+1]:
                break
        print(maxCycle[i],maxCycle[i+1],target[j][2]["weight"])
        weightList.append(target[j][2]["weight"])
    else:
        for j in range(len(target)):
            if target[j][1]==maxCycle[0]:
                break
        print(maxCycle[i],maxCycle[0],target[j][2]["weight"])
        weightList.append(target[j][2]["weight"])

In [ ]:
list(itertools.permutations([1, 2, 3]))

In [15]:
roterSetList=list(itertools.permutations([roter_1, roter_2, roter_3]))
minWeight=min(weightList)
relativeWeight = list(map(lambda x: x-minWeight, weightList))

In [ ]:
relativeWeight

In [17]:
def bombe(i):
    bang=0
    offset=26*27+1
    cycleLength=len(maxCycle)
    roter=roterSetList[i]
    for h in tqdm.tqdm(range(17576)):
        counter=offset+h
        position=list(num2alpha(counter))
        for j in range(17576):
            bang=1
            ring=list(num2alpha(offset+j))
            for k in range(cycleLength):
                if k!=cycleLength-1:
                    if enigmaEncode(maxCycle[k],roter,position,ring,offset=relativeWeight[k])!=maxCycle[k+1]:
                        bang*=0
                else:
                    if enigmaEncode(maxCycle[k],roter,position,ring,offset=relativeWeight[k])!=maxCycle[0]:
                        bang*=0
                if bang==0:
                    break
            if bang==1:
                print("Bang!!!",i,position,ring)
                bang=0
                return(i,position,ring)

In [ ]:
i,position,ring=bombe(2)

In [ ]:
enigmaEncode(enc,roterSetList[i],position,ring,-minWeight+1)